In [1]:
from datetime import date, time, datetime
from google.oauth2 import service_account
from google.cloud import bigquery, storage
import pandas as pd
import json

In [2]:
dev = True
if (dev):
    key_path = r"C:\Users\nites\OneDrive\Documents\napo-nitesh-local-ae32-vpcservice-datawarehouse-879a160a28ee.json"
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

    client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
    storage_client = storage.Client(credentials=credentials, project=credentials.project_id,) 

else: 
    client = bigquery.Client()
    storage_client = storage.Client()

In [9]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    autodetect=True,
    max_bad_records=1,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

In [4]:
bucket_name='data-warehouse-harbour'
dataset_name='postgres'
path_name='policy-service/{}'.format(datetime.now().strftime("%Y-%m-%d"))
bucket = storage_client.get_bucket(bucket_name)

In [5]:
table_mapping = {
    "policy.breed.json":"breed",
    "policy.customer.json":"customer",
    "policy.pet.json":"pet",
    "policy.policy.json":"policy",
    "policy.product.json":"product",
    "policy.productline.json":"productline",
    "policy.payment.json":"policy_payment"
}

In [12]:
#print(table_mapping,table_id,uri,table_mapping["policy.customer.json"])

In [10]:
for table in table_mapping:
    uri = "gs://{}/{}/{}".format(bucket_name,path_name,table)
    table_id = "ae32-vpcservice-datawarehouse.{}.{}".format(dataset_name,table_mapping[table])
    load_job = client.load_table_from_uri(
        uri,
        table_id,
        location="EU",  # To match the destination dataset location
        job_config=job_config,
        
    )
    print(table,load_job.result())


policy.customer.json <google.cloud.bigquery.job.load.LoadJob object at 0x000001C4DAD4EB80>


In [ ]:
# Subscription table loading
blob = bucket.blob("{}/{}".format(path_name,'policy.subscription.json'))
df = pd.read_json(blob.download_as_string(), lines=True)
df1 = pd.json_normalize(df['fields'])

,_policyid,active,created_date,modified_date,failed_payments,failed_payment_events
0,3009,True,2022-05-25T15:38:57.527Z,2022-06-29T15:33:31.961Z,[],[]
1,20146,True,2022-07-19T16:56:31.226Z,2022-07-19T16:56:31.226Z,[],[]
2,217,True,2022-02-07T10:49:07.615Z,2022-06-29T15:32:50.299Z,[],[]
3,224,True,2022-02-07T10:49:07.615Z,2022-06-29T15:32:50.316Z,[],[]
4,225,True,2022-02-07T10:49:07.615Z,2022-06-29T15:32:50.318Z,[],[]


In [ ]:
if 'policyid' in df1.columns:
    policy_column = 'policyid' 
if '_policyid' in df1.columns:
    policy_column = '_policyid'
    df1.rename(columns={'_policyid':'policyid'},inplace=True)

In [ ]:
if (df1.policyid.count()==df.model.count()):
    print('True')
    df_final = pd.merge(df, df1, left_index=True, right_index=True)
    df_final.drop(columns=['fields','failed_payment_events'],inplace=True)

job_config2 = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

load_job = client.load_table_from_dataframe(
    df_final
    ,"ae32-vpcservice-datawarehouse.{}.subscription".format(dataset_name)
    ,job_config=job_config2
    ,location='EU'
) 
print(load_job.result())

True
